## Read input

Today's input required quite a few steps to parse but those steps were rather straight-forward to me at least. In the first section, I parse the workflows by separating rules that have a condition and ones that don't (and create a custom condition of `*` for them. Having uniform structure for all rules makes the logic itself bit simpler.

For ratings, I remove the curly braces and split to pairs and store them in a list of dictionaries.


In [1]:
from utils import read_multisection_input
import re

def workflows(section):
    rules = {}
    for line in section.split('\n'):
        key, rest = line.split('{')
        rest = rest.split('}')[0].split(',')
        tests = []
        for rule in rest:
            if ':' in rule:
                condition, target = rule.split(':')
                attribute, operator, value = re.findall(r'(x|m|a|s)(<|>)(\d+)', condition)[0]
                condition = (attribute, operator, int(value))
            else:
                condition = '*'
                target = rule
            tests.append((condition, target))
        rules[key] = tests
    return rules
        
def ratings(section):
    ratings_output = []
    for line in section.split('\n'):
        line = line.replace('{', '').replace('}', '')
        ratings = line.split(',')
        r = {}
        for rating in ratings:
            key, value = rating.split('=')
            value = int(value)
            r[key] = value
        ratings_output.append(r.copy())
    return ratings_output
        
workflows, ratings = read_multisection_input(19, [workflows, ratings])

To sort the matchines, I run each one against a loop of conditions, following the path it leads them in the workflows.

Once we hit one where the target is 'A' or 'R', we store them in the final bucket and move to the next machine. This will result with a dictionary of two lists: one for accepted and one for rejected machines.

In [2]:
from collections import defaultdict

FINAL_TARGETS = ('A', 'R')


def sort_machines(workflows, machines):
    BEGIN = 'in'
    buckets = defaultdict(list)

    for machine in machines:
        current = workflows[BEGIN]
        i = 0
        while True:
            condition, target = current[i]
            match condition:
                case '*':
                    if target in FINAL_TARGETS:
                        buckets[target].append(machine)
                        break
                    else:
                        current = workflows[target]
                        i = 0
                        continue
                case (attribute, '<', value):
                    if machine[attribute] < value:
                        if target in FINAL_TARGETS:
                            buckets[target].append(machine)
                            break
                        else:
                            current = workflows[target]
                            i = 0
                            continue
                case (attribute, '>', value):
                    if machine[attribute] > value:
                        if target in FINAL_TARGETS:
                            buckets[target].append(machine)
                            break
                        else:
                            current = workflows[target]
                            i = 0
                            continue
            i+=1
    return buckets

To calculate the final result, sum all the values of all the machines in the 'A' bucket.

In [3]:
def calculate_accepted_score(buckets):
    return sum(sum(machine.values()) for machine in buckets['A'])

In [4]:
buckets = sort_machines(workflows, ratings)
part_1 = calculate_accepted_score(buckets)

print(f'Solution: {part_1}')
assert part_1 == 280909

Solution: 280909


## One star today

I'm leaving today to one star for now. I had a busy day today with my local communities (wrapping up one and kickstarting another) and there's hockey on TV so I'm gonna chill for the night. At the time of writing, my total count is 35 stars (missing both stars of day 17 and today's 2nd star out of possible 38).